In [3]:
import tiktoken
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
import pandas as pd

In [4]:
def split_table_into_chunks(table_markdown: str, max_tokens: int = 700, encoding_name: str = "cl100k_base") -> list:
    """
    Splits a Markdown table into smaller chunks, ensuring the total number of tokens 
    does not exceed `max_tokens`.
    
    Parameters:
    - table_markdown (str): The Markdown string containing the table to be split.
    - max_tokens (int): Maximum number of tokens per chunk (default is 700).
    - encoding_name (str): Name of the token encoding used by `tiktoken` (default is "cl100k_base").
    
    Returns:
    - List[str]: A list of Markdown table chunks.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    lines = table_markdown.strip().split("\n")
    chunks = []
    current_chunk = []
    current_tokens = 0
    
    for line in lines:
        line_tokens = len(encoding.encode(line))
        if current_tokens + line_tokens > max_tokens:
            chunks.append("\n".join(current_chunk))
            current_chunk = [line]
            current_tokens = line_tokens
        else:
            current_chunk.append(line)
            current_tokens += line_tokens
    
    if current_chunk:
        chunks.append("\n".join(current_chunk))
    
    return chunks

In [5]:
# Get client LLMs
load_dotenv()
deployment_name = os.getenv("GPT4O__DEPLOYMENT_NAME")
client = AzureOpenAI(
    api_key=os.environ["GPT4O__KEY"],
    azure_endpoint=os.environ["GPT4O__ENDPOINT"],
    api_version=os.environ["GPT4O__API_VERSION"],
)

In [6]:
# System Prompt for extracting entities
SYSTEM_PROMPT = """
"""

# User Prompt for extracting entities
USER_PROMPT = """
-Goal-
Given a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: One of the following types: [{entity_types}]
- entity_description: Provide an in-depth explanation of the entity’s attributes and activities.
  **(Include all known attributes, actions, behaviors, relationships with other entities or events, and any context from the text that helps describe the entity in a comprehensive manner. Ensure that every relevant detail from the input text is incorporated into the appropriate entity_descriptions, so that no portion of the text is left out. If there are additional facts, observations, or details in the text that do not appear to fit neatly under any existing entity, either create a new entity for them or expand an existing entity’s description to capture them.)**

Format each entity as ("entity"{tuple_delimiter}"<entity_name>"{tuple_delimiter}"<entity_type>"{tuple_delimiter}"<entity_description>")

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other
- relationship_strength: a numeric score indicating strength of the relationship between the source entity and target entity
 Format each relationship as ("relationship"{tuple_delimiter}"<source_entity>"{tuple_delimiter}"<target_entity>"{tuple_delimiter}"<relationship_description>"{tuple_delimiter}"<relationship_strength>")

3. Return output in Vietnamese as a single list of all the entities and relationships identified in steps 1 and 2. Use **{record_delimiter}** as the list delimiter.

4. When finished, output {completion_delimiter}

5. Double-check that every detail from the input text is represented in either an entity description or a relationship description. If any piece of information is missing, revise the entity descriptions or create new entities/relationships until you have accounted for all details.

######################
-Examples-
######################
Example 1:

Entity_types: [person, technology, mission, organization, location]
Text:
3. Ghi nhận thời gian làm việc

Làm việc tại văn phòng:

Công ty sử dụng công cụ xử lý dữ liệu hệ thống bằng thẻ từ, sau đây gọi tắt là máy chấm công để theo dõi thời gian ra, vào nơi làm việc của CBNV.

Mỗi CBNV được cung cấp 01 thẻ từ để ghi lại thời gian làm việc tại Công ty.

CBNV phải quẹt thẻ tại máy chấm công khi bắt đầu làm việc và trước khi ra về. Hệ thống cho phép đồng bộ dữ liệu quẹt thẻ ở toàn bộ văn phòng của Sun*. Trường hợp đi công tác hoặc di chuyển giữa các văn phòng, CBNV đều có thể sử dụng thẻ nhân viên của mình để quẹt và hệ thống ghi nhận giờ quẹt thẻ đầu tiên làm giờ in và giờ quẹt thẻ cuối cùng làm giờ out.

CBNV chỉ được phép dùng thẻ của mình để quẹt thẻ chấm công, không được quẹt thẻ hộ người khác. Trường hợp bị phát hiện quẹt thẻ chấm công hộ người khác, thì người quẹt hộ và người nhờ quẹt hộ sẽ bị áp dụng hình thức khiển trách nội bộ và bị giảm trừ 1.000.000 VNĐ cho mỗi lần vi phạm vào kỳ thưởng hiệu suất giữa năm hoặc thưởng tháng lương thứ 13.

Trường hợp quên thẻ, CBNV làm đơn trên hệ thống và xin xác nhận của CBQL ngay trong ngày phát sinh quên thẻ. CBQL có trách nhiệm xác nhận và phê duyệt đơn ngay sau thời điểm nhận được thông báo từ CBNV.

Trường hợp quên quẹt thẻ, CBNV làm đơn trên hệ thống và xin xác nhận của CBQL trong 03 ngày làm việc kể từ ngày quên quẹt thẻ.

Các trường hợp quên thẻ/quên quẹt thẻ mà không có sự xác nhận từ CBQL theo quy định dẫn đến tình trạng bị thiếu dữ liệu, CBNV sẽ bị tính nghỉ không lương cả ngày hoặc nửa ngày tuỳ vào dữ liệu chấm công thực tế.
################
Output:
("entity"{tuple_delimiter}"CÔNG TY"{tuple_delimiter}"organization"{tuple_delimiter}"Công ty sử dụng công cụ xử lý dữ liệu hệ thống bằng thẻ từ để theo dõi thời gian làm việc của cán bộ nhân viên (CBNV). Công ty áp dụng quy định về việc sử dụng thẻ từ để đảm bảo quản lý chặt chẽ thời gian ra vào làm việc và tính toán các trường hợp vi phạm liên quan đến thời gian làm việc."){record_delimiter}
("entity"{tuple_delimiter}"CBNV"{tuple_delimiter}"person"{tuple_delimiter}"Cán bộ nhân viên (CBNV) là đối tượng thực hiện các quy định về việc sử dụng thẻ từ để ghi nhận thời gian làm việc. CBNV có trách nhiệm quẹt thẻ tại máy chấm công khi bắt đầu làm việc và ra về. CBNV cũng phải chịu các chế tài nếu vi phạm quy định liên quan."){record_delimiter}
("entity"{tuple_delimiter}"MÁY CHẤM CÔNG"{tuple_delimiter}"technology"{tuple_delimiter}"Máy chấm công là công cụ được sử dụng để ghi nhận thời gian ra vào nơi làm việc của CBNV bằng cách quẹt thẻ từ. Máy này đồng bộ dữ liệu toàn bộ văn phòng và hỗ trợ việc ghi nhận dữ liệu chính xác."){record_delimiter}
("entity"{tuple_delimiter}"THẺ TỪ"{tuple_delimiter}"technology"{tuple_delimiter}"Thẻ từ là công cụ cá nhân được cấp cho mỗi CBNV để ghi lại thời gian làm việc tại Công ty. Thẻ từ được sử dụng khi quẹt tại máy chấm công và đảm bảo tính cá nhân hóa trong việc theo dõi thời gian làm việc."){record_delimiter}
("entity"{tuple_delimiter}"CBQL"{tuple_delimiter}"person"{tuple_delimiter}"Cán bộ quản lý (CBQL) có trách nhiệm xác nhận và phê duyệt đơn xin quên thẻ/quên quẹt thẻ của CBNV. CBQL đóng vai trò quan trọng trong việc xác nhận thông tin liên quan đến các trường hợp phát sinh lỗi trong ghi nhận thời gian làm việc."){record_delimiter}
("entity"{tuple_delimiter}"VĂN PHÒNG SUN*"{tuple_delimiter}"location"{tuple_delimiter}"Địa điểm làm việc nơi hệ thống máy chấm công và thẻ từ được áp dụng. Văn phòng hỗ trợ việc đồng bộ dữ liệu quẹt thẻ trên toàn hệ thống của Sun*."){record_delimiter}
("entity"{tuple_delimiter}"QUY ĐỊNH VỀ THẺ TỪ"{tuple_delimiter}"mission"{tuple_delimiter}"Quy định về việc sử dụng thẻ từ yêu cầu CBNV phải quẹt thẻ tại máy chấm công khi bắt đầu làm việc và trước khi ra về. Vi phạm quy định này sẽ dẫn đến chế tài kỷ luật và giảm thưởng."){record_delimiter}
("entity"{tuple_delimiter}"CHẾ TÀI VI PHẠM"{tuple_delimiter}"mission"{tuple_delimiter}"Công ty áp dụng chế tài nghiêm khắc đối với các vi phạm liên quan đến việc sử dụng thẻ từ, bao gồm khiển trách nội bộ và giảm trừ 1.000.000 VNĐ vào kỳ thưởng hiệu suất giữa năm hoặc thưởng lương tháng 13."){record_delimiter}
("entity"{tuple_delimiter}"ĐƠN XIN QUÊN THẺ"{tuple_delimiter}"technology"{tuple_delimiter}"Đơn xin được thực hiện trên hệ thống khi CBNV quên thẻ. CBNV cần xin xác nhận của CBQL ngay trong ngày phát sinh lỗi."){record_delimiter}
("entity"{tuple_delimiter}"ĐƠN XIN QUÊN QUẸT THẺ"{tuple_delimiter}"technology"{tuple_delimiter}"Đơn được thực hiện trên hệ thống khi CBNV quên quẹt thẻ. Cần được nộp và xác nhận bởi CBQL trong vòng 03 ngày làm việc kể từ ngày phát sinh lỗi."){record_delimiter}
("entity"{tuple_delimiter}"DỮ LIỆU CHẤM CÔNG"{tuple_delimiter}"technology"{tuple_delimiter}"Hệ thống dữ liệu ghi nhận giờ vào và giờ ra của CBNV, được đồng bộ trên toàn bộ văn phòng. Các lỗi phát sinh không có xác nhận của CBQL sẽ dẫn đến tình trạng bị tính nghỉ không lương."){record_delimiter}
("relationship"{tuple_delimiter}"CÔNG TY"{tuple_delimiter}"MÁY CHẤM CÔNG"{tuple_delimiter}"Công ty sử dụng máy chấm công để theo dõi thời gian ra vào làm việc của CBNV, hỗ trợ quản lý hệ thống dữ liệu chấm công."{tuple_delimiter}"9"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"THẺ TỪ"{tuple_delimiter}"Mỗi CBNV được cung cấp một thẻ từ để ghi lại thời gian làm việc và phải sử dụng thẻ này cho việc quẹt thẻ chấm công."{tuple_delimiter}"9"){record_delimiter}
("relationship"{tuple_delimiter}"THẺ TỪ"{tuple_delimiter}"MÁY CHẤM CÔNG"{tuple_delimiter}"Thẻ từ được sử dụng tại máy chấm công để ghi nhận giờ làm việc của CBNV, với dữ liệu được đồng bộ toàn hệ thống."{tuple_delimiter}"8"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"CBQL"{tuple_delimiter}"CBQL có trách nhiệm xác nhận đơn từ liên quan đến các trường hợp quên thẻ/quên quẹt thẻ của CBNV, đảm bảo dữ liệu được cập nhật chính xác."{tuple_delimiter}"7"){record_delimiter}
("relationship"{tuple_delimiter}"CBQL"{tuple_delimiter}"CÔNG TY"{tuple_delimiter}"CBQL thực hiện vai trò quản lý, đảm bảo thực hiện các quy định của Công ty về việc ghi nhận thời gian làm việc."{tuple_delimiter}"8"){record_delimiter}
("relationship"{tuple_delimiter}"VĂN PHÒNG SUN*"{tuple_delimiter}"THẺ TỪ"{tuple_delimiter}"Thẻ từ có thể sử dụng tại tất cả văn phòng Sun* để đồng bộ dữ liệu quẹt thẻ giữa các văn phòng."{tuple_delimiter}"7"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"QUY ĐỊNH VỀ THẺ TỪ"{tuple_delimiter}"CBNV phải tuân thủ quy định sử dụng thẻ từ, bao gồm việc quẹt thẻ đúng quy định khi bắt đầu làm việc và trước khi ra về."{tuple_delimiter}"9"){record_delimiter}
("relationship"{tuple_delimiter}"QUY ĐỊNH VỀ THẺ TỪ"{tuple_delimiter}"CHẾ TÀI VI PHẠM"{tuple_delimiter}"Vi phạm quy định về sử dụng thẻ từ sẽ dẫn đến áp dụng các chế tài kỷ luật nghiêm khắc."{tuple_delimiter}"8"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"ĐƠN XIN QUÊN THẺ"{tuple_delimiter}"CBNV phải thực hiện đơn xin quên thẻ trên hệ thống khi xảy ra tình trạng quên mang thẻ từ."{tuple_delimiter}"7"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"ĐƠN XIN QUÊN QUẸT THẺ"{tuple_delimiter}"CBNV có trách nhiệm làm đơn xin xác nhận quên quẹt thẻ trong vòng 03 ngày làm việc để tránh bị tính nghỉ không lương."{tuple_delimiter}"7"){record_delimiter}
("relationship"{tuple_delimiter}"CBQL"{tuple_delimiter}"ĐƠN XIN QUÊN THẺ"{tuple_delimiter}"CBQL chịu trách nhiệm xác nhận và phê duyệt đơn xin quên thẻ của CBNV trong ngày phát sinh."{tuple_delimiter}"8"){record_delimiter}
("relationship"{tuple_delimiter}"CBQL"{tuple_delimiter}"ĐƠN XIN QUÊN QUẸT THẺ"{tuple_delimiter}"CBQL có trách nhiệm xác nhận đơn xin quên quẹt thẻ của CBNV trong vòng 03 ngày làm việc."{tuple_delimiter}"8"){record_delimiter}
("relationship"{tuple_delimiter}"DỮ LIỆU CHẤM CÔNG"{tuple_delimiter}"MÁY CHẤM CÔNG"{tuple_delimiter}"Dữ liệu chấm công được ghi nhận từ máy chấm công, đảm bảo đồng bộ trên toàn hệ thống văn phòng."{tuple_delimiter}"9"){record_delimiter}
("relationship"{tuple_delimiter}"DỮ LIỆU CHẤM CÔNG"{tuple_delimiter}"CBNV"{tuple_delimiter}"Dữ liệu chấm công phản ánh giờ vào và ra của CBNV và là cơ sở tính toán thời gian làm việc."{tuple_delimiter}"9"){record_delimiter}{completion_delimiter}
#############################
Example 2:

Entity_types: [person, technology, mission, organization, location]
Text:
Điều 5: Quy định về làm thêm giờ

- CBNV phát sinh làm thêm giờ hoặc làm việc tự nguyện ngày thứ Bảy phải điền vào Đơn yêu cầu làm thêm ngoài giờ/Làm việc ngày thứ Bảy trên hệ thống quản lý thời gian làm việc WSM, đồng thời phải được sự phê duyệt của CBQL trước thời hạn chốt công hàng tháng.

- Tổng thời gian làm thêm giờ không quá 40 giờ/tháng và 200 giờ/năm hoặc không quá 300 giờ trong một năm trong các trường hợp theo quy định của pháp luật lao động.

- Cách thức ghi nhận thời gian làm thêm giờ trong tháng được dựa trên hai cơ sở dữ liệu:
    - Thời gian theo đơn Yêu cầu làm thêm giờ được sự đồng ý của CBQL.
    - Thời gian quẹt thẻ thực tế: dựa vào dữ liệu quẹt thẻ thực tế được ghi nhận trên hệ thống wsm.

- Nếu thời gian đăng ký trong đơn lớn hơn thời gian quẹt thẻ thực tế: hệ thống ghi nhận thời gian OT là thời gian quẹt thẻ thực tế.

- Nếu thời gian đăng ký trong đơn nhỏ hơn thời gian quẹt thẻ thực tế: hệ thống ghi nhận thời gian OT là thời gian đăng ký trong đơn. CBNV có trách nhiệm bổ sung đơn đăng ký cho khoảng thời gian chênh lệch trên, đồng thời đơn này cũng phải được sự phê duyệt của CBQL.

- Trong trường hợp phát sinh OT vào ngày cuối cùng của tháng (ngày 23) thì CBNV phải hoàn thiện đơn đăng ký có xác nhận của CBQL trên hệ thống chậm nhất vào 23:59’ ngày hôm đó.

- Nếu thời gian OT kéo dài qua đêm, sẽ phải thực hiện làm hai đơn đăng ký cho hai ngày.

- Thời gian làm thêm với các ngày từ thứ Hai đến thứ Sáu được tính sau 01 giờ kể từ khi kết thúc ca làm việc (muộn nhất là 18:45).
#############
Output:
("entity"{tuple_delimiter}"CBNV"{tuple_delimiter}"person"{tuple_delimiter}"CBNV là viết tắt của cán bộ nhân viên, đại diện cho những người lao động trong tổ chức thực hiện các nhiệm vụ, có trách nhiệm đăng ký làm thêm giờ theo đúng quy định của công ty và pháp luật."){record_delimiter}
("entity"{tuple_delimiter}"CBQL"{tuple_delimiter}"person"{tuple_delimiter}"CBQL là viết tắt của cán bộ quản lý, là người có thẩm quyền phê duyệt các yêu cầu làm thêm giờ của CBNV và đảm bảo tuân thủ các quy định quản lý thời gian làm việc."){record_delimiter}
("entity"{tuple_delimiter}"WSM"{tuple_delimiter}"technology"{tuple_delimiter}"WSM là hệ thống quản lý thời gian làm việc, được sử dụng để ghi nhận và quản lý thời gian làm thêm giờ của CBNV thông qua việc đăng ký và quẹt thẻ thực tế."){record_delimiter}
("entity"{tuple_delimiter}"Thời gian làm thêm giờ"{tuple_delimiter}"mission"{tuple_delimiter}"Thời gian làm thêm giờ đề cập đến khoảng thời gian làm việc vượt ngoài giờ hành chính, phải được đăng ký và phê duyệt theo quy định cụ thể, không vượt quá giới hạn pháp luật (40 giờ/tháng, 200 hoặc 300 giờ/năm)."){record_delimiter}
("entity"{tuple_delimiter}"Quẹt thẻ thực tế"{tuple_delimiter}"technology"{tuple_delimiter}"Quẹt thẻ thực tế là hành động sử dụng thẻ để ghi nhận thời gian làm việc thực tế của CBNV trên hệ thống WSM, là cơ sở để đối chiếu với thời gian đăng ký làm thêm giờ."){record_delimiter}
("entity"{tuple_delimiter}"Pháp luật lao động"{tuple_delimiter}"organization"{tuple_delimiter}"Pháp luật lao động là hệ thống quy định pháp lý điều chỉnh các vấn đề liên quan đến lao động, bao gồm giới hạn thời gian làm thêm giờ và các quy định liên quan đến việc làm thêm."){record_delimiter}
("entity"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"technology"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ là tài liệu được CBNV sử dụng để đăng ký thời gian làm thêm giờ trên hệ thống WSM, phải được sự phê duyệt của CBQL trước thời hạn chốt công."){record_delimiter}
("entity"{tuple_delimiter}"Ngày thứ Bảy"{tuple_delimiter}"mission"{tuple_delimiter}"Ngày thứ Bảy là thời gian không thuộc ngày làm việc chính thức trong tuần, được quy định cho việc làm việc tự nguyện hoặc làm thêm giờ, cần đăng ký và được phê duyệt."){record_delimiter}
("entity"{tuple_delimiter}"Thời gian OT kéo dài qua đêm"{tuple_delimiter}"mission"{tuple_delimiter}"Thời gian OT kéo dài qua đêm là khoảng thời gian làm thêm giờ liên tục từ ngày này sang ngày khác, đòi hỏi phải thực hiện hai đơn đăng ký riêng biệt cho từng ngày."){record_delimiter}
("entity"{tuple_delimiter}"Ngày 23"{tuple_delimiter}"time"{tuple_delimiter}"Ngày 23 được quy định là ngày cuối cùng của tháng, là thời điểm cuối cùng để CBNV hoàn tất việc đăng ký làm thêm giờ và được CBQL phê duyệt trước hạn chốt công."){record_delimiter}
("entity"{tuple_delimiter}"Thời gian 01 giờ"{tuple_delimiter}"time"{tuple_delimiter}"Thời gian 01 giờ là khoảng thời gian đệm sau khi ca làm việc kết thúc, từ đó mới bắt đầu tính thời gian làm thêm giờ từ thứ Hai đến thứ Sáu."){record_delimiter}
("entity"{tuple_delimiter}"18:45"{tuple_delimiter}"time"{tuple_delimiter}"18:45 là thời gian muộn nhất được quy định để tính giờ làm thêm trong ngày làm việc bình thường (thứ Hai đến thứ Sáu)."){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"CBQL"{tuple_delimiter}"CBNV phải được sự phê duyệt của CBQL khi đăng ký làm thêm giờ, thể hiện mối quan hệ quản lý và giám sát giữa nhân viên và cấp quản lý."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"WSM"{tuple_delimiter}"CBNV sử dụng hệ thống WSM để đăng ký và ghi nhận thời gian làm thêm giờ, cho thấy mối quan hệ tương tác giữa người lao động và công nghệ quản lý thời gian."{tuple_delimiter}"5"){record_delimiter}
("relationship"{tuple_delimiter}"WSM"{tuple_delimiter}"Quẹt thẻ thực tế"{tuple_delimiter}"WSM ghi nhận thời gian làm thêm giờ dựa trên dữ liệu từ quẹt thẻ thực tế, thể hiện mối liên hệ trực tiếp giữa công nghệ và hành động thực tế."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Thời gian làm thêm giờ"{tuple_delimiter}"Pháp luật lao động"{tuple_delimiter}"Giới hạn thời gian làm thêm giờ được quy định bởi pháp luật lao động, thể hiện sự điều chỉnh và kiểm soát của pháp luật đối với hành vi làm thêm giờ."{tuple_delimiter}"5"){record_delimiter}
("relationship"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"CBQL"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ phải được CBQL phê duyệt, thể hiện sự phụ thuộc của tài liệu vào quyết định của cấp quản lý."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"WSM"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ được nộp và quản lý trên hệ thống WSM, cho thấy sự tích hợp giữa tài liệu và công nghệ quản lý."{tuple_delimiter}"5"){record_delimiter}
("relationship"{tuple_delimiter}"CBNV"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"CBNV phải điền vào đơn yêu cầu làm thêm giờ khi phát sinh làm việc ngoài giờ, thể hiện sự tuân thủ quy định của nhân viên với hệ thống quản lý."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Quẹt thẻ thực tế"{tuple_delimiter}"Thời gian làm thêm giờ"{tuple_delimiter}"Thời gian quẹt thẻ thực tế là một trong hai cơ sở để ghi nhận thời gian làm thêm giờ, thể hiện sự liên kết giữa hành động thực tế và thời gian làm thêm được tính toán."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Ngày thứ Bảy"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"Khi làm việc vào ngày thứ Bảy, CBNV cần điền đơn yêu cầu làm thêm giờ và được CBQL phê duyệt, thể hiện mối liên hệ giữa quy định làm việc và tài liệu quản lý."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Thời gian OT kéo dài qua đêm"{tuple_delimiter}"Đơn yêu cầu làm thêm giờ"{tuple_delimiter}"Thời gian OT kéo dài qua đêm đòi hỏi phải thực hiện hai đơn đăng ký riêng biệt, liên kết chặt chẽ với quy định tài liệu."{tuple_delimiter}"5"){record_delimiter}
("relationship"{tuple_delimiter}"Ngày 23"{tuple_delimiter}"CBNV"{tuple_delimiter}"CBNV phải hoàn thành đơn yêu cầu OT trước 23:59 ngày 23, thể hiện mối quan hệ giữa thời gian quy định và trách nhiệm của nhân viên."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Ngày 23"{tuple_delimiter}"CBQL"{tuple_delimiter}"CBQL cần phê duyệt đơn OT của CBNV trước 23:59 ngày 23, cho thấy vai trò quản lý trong kiểm soát thời gian làm thêm."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"Thời gian 01 giờ"{tuple_delimiter}"Thời gian làm thêm giờ"{tuple_delimiter}"Thời gian 01 giờ là thời gian chờ bắt đầu tính làm thêm giờ, được quy định cụ thể trong phạm vi thời gian làm thêm giờ."{tuple_delimiter}"4"){record_delimiter}
("relationship"{tuple_delimiter}"18:45"{tuple_delimiter}"Thời gian làm thêm giờ"{tuple_delimiter}"18:45 là thời gian muộn nhất để tính thời gian làm thêm trong ngày làm việc, liên kết với quy định về giờ làm thêm."{tuple_delimiter}"4"){record_delimiter}{completion_delimiter}
#############################

-Real Data-
######################
Entity_types: {entity_types}
Text: {input_text}
######################
Output:
"""

In [7]:
# System Prompt for extracting entities
SYSTEM_PROMPT_SUM = """
You are an AI specialized in text analysis and summarization. Your task is to extract the main content of a specific section within a Markdown table while ensuring a concise and context-aware summary.
"""

# User Prompt for extracting entities
USER_PROMPT_SUM = """
You are an AI specialized in text analysis and summarization. Your task is to extract the main content of a specific section within a Markdown table while ensuring a concise and context-aware summary.

1. Output requirements:
You must ensure the following:

Main topic and core content.
The overview of the topic does not need to be too detailed. But it is necessary to extract important words to represent the topic being processed
Summarize previously mentioned content to provide necessary context.
Concise and focused content to summarize the context
Update or expand the summary if necessary, avoiding irrelevant or far-fetched information.
Output is in Vietnamese.

2. Input format:
Filename: {file_name}
Previous summary: {previous_summary}
Current chunk: {input_chunk}

3. Output format:
Document references to [Topic], [Overview of that topic based on previous information]

4. Key considerations:
Extract only the information currently being processed in the chunk.
Avoid including content that is too far-fetched or irrelevant.

5. Output:
"""

In [24]:
# Settings inputs 
big_table = """
|**PHẦN I <br>QUY ĐỊNH VỀ VIỆC CÀI ĐẶT PHẦN MỀM<br>(Áp dụng đối với tất cả các thiết bị liên quan đến quá trình xử lý & lưu trữ thông tin: máy tính bàn, laptop, thiết bị di động,…của Công ty, cũng như của nhân viên/ khách hàng được phép kết nối vào mạng nội bộ Công ty)**|**PHẦN I <br>QUY ĐỊNH VỀ VIỆC CÀI ĐẶT PHẦN MỀM<br>(Áp dụng đối với tất cả các thiết bị liên quan đến quá trình xử lý & lưu trữ thông tin: máy tính bàn, laptop, thiết bị di động,…của Công ty, cũng như của nhân viên/ khách hàng được phép kết nối vào mạng nội bộ Công ty)**|
|---|---|
| |Nghiêm cấm tự ý cài đặt, sử dụng phần mềm trong danh mục phần mềm rủi ro cao, bao gồm nhưng không giới hạn các nhóm phần mềm sau:- Phần mềm điều khiển máy từ xa;<br>- Phần mềm quay/chụp màn hình, dịch vụ đọc, chia sẻ file, source code mà có tính năng lưu trữ trên Internet: Pastebin, Json Viewer, Gyazo,...;<br>- Phần mềm Proxy, OpenVPN, HotspotShield,... để vượt hệ thống Security của Công ty;<br>- Phần mềm theo dõi, thu thập dữ liệu trái phép: Keylogger,...;<br>- Cài đặt máy ảo Kali Linux, Backtrack,...;<br>|
| |[*Tham khảo: Phần I - Danh sách các phần mềm rủi ro cao (không được phép)*](None)|
| |Nghiêm cấm các hành vi gây ảnh hưởng đến băng thông của Công ty (như cài đặt các phần mềm tăng tốc download, phần mềm chia sẻ P2P,...);|
| |Nghiêm cấm các hành vi, các thủ thuật tấn công hệ thống của Công ty và người dùng;|
| |[*Tham khảo: Phần II - Danh sách các hành vi/ thủ thuật không được phép*](None)|
| |Không được phép tự ý cài đặt các phần mềm không có bản quyền, phần mềm giải trí không liên quan đến công việc (VD: các phần mềm game...);|
| |Không được phép crack tất cả các phần mềm yêu cầu bản quyền (Photoshop, Microsoft Office,...);|
| |[*Tham khảo: PL02 - Danh sách phần mềm yêu cầu bản quyền hợp pháp*](None)|
| |Cài đặt Antivirus và bật Firewall trên tất cả các thiết bị máy tính (Windows, MAC, Ubuntu);|
| |[Tham khảo: Hướng dẫn cài đặt phần mềm Antivirus](https://docs.google.com/document/d/1RfSo4zGVDjp1CLZXQ0--StiRWeEDzSU1uwiLEk47VL8/edit)|
| |Không được phép vô hiệu hóa, gỡ bỏ hay thay đổi cài đặt VPN, các phần mềm an toàn như phần mềm mã hóa, phần mềm diệt virus, firewall,...;|
| |Nghiêm cấm các hình thức: cài lại Hệ điều hành, tự ý nâng cấp Hệ điều hành hoặc cài các Hệ điều hành khác.|
|**PHẦN II <br>QUY ĐỊNH CHUNG ĐỐI VỚI NHÂN VIÊN TẠI CÔNG TY**|**PHẦN II <br>QUY ĐỊNH CHUNG ĐỐI VỚI NHÂN VIÊN TẠI CÔNG TY**|
| |Đeo thẻ trong khu vực thuộc phạm vi Công ty; |
| |Khóa màn hình trước khi rời khỏi vị trí làm việc;|
| |Không được phép sử dụng các thiết bị lưu trữ thông tin của cá nhân (usb, thẻ nhớ, ổ cứng,...) xuất dữ liệu thông tin liên quan đến công việc khi chưa được cho phép;|
| |Nhân viên sau khi sử dụng phòng họp, phải chú ý:<br>- Xóa các thông tin trên bảng <br>- Thu dọn/hủy các giấy tờ chứa các thông tin liên quan<br>- Đặt lại các thiết bị tại vị trí như ban đầu<br>- Tắt các thiết bị không sử dụng|
| |Nghiêm cấm sử dụng tài khoản cá nhân (email, GG Driver..) cho mục đích công việc. <br>Mọi hoạt động liên quan đến công việc đều phải được thực hiện trên tài khoản GSuite của Công ty cấp cho nhân viên.|
|**PHẦN III<br>QUY ĐỊNH VỀ VIỆC QUẢN LÝ CÁC THIẾT BỊ CỦA CÔNG TY <br>(Áp dụng với thiết bị liên quan đến quá trình xử lý & lưu trữ thông tin (máy tính bàn, laptop, di động,…) do Công ty cung cấp)**|**PHẦN III<br>QUY ĐỊNH VỀ VIỆC QUẢN LÝ CÁC THIẾT BỊ CỦA CÔNG TY <br>(Áp dụng với thiết bị liên quan đến quá trình xử lý & lưu trữ thông tin (máy tính bàn, laptop, di động,…) do Công ty cung cấp)**|
| |Tắt hoàn toàn (Shut down) tất cả các thiết bị máy tính thuộc quản lý của nhân viên sau khi kết thúc ngày làm việc; <br>(Trừ một số thiết bị chuyên biệt của ban IFU - ban Hạ tầng và hệ thống và 1 số trường hợp bắt buộc phải bật để phục vụ cho công việc)|
| |Tất cả các thiết bị (máy tính, thiết bị test, adapter macbook...) phải được lưu trữ trong Cabinet của nhân viên sau khi kết thúc ngày làm việc.|
| |Không được phép cho người không liên quan sử dụng các thiết bị có chứa các dữ liệu của dự án, của Công ty.|
| |Không được phép mang các thiết bị lưu trữ thông tin của Công ty (thiết bị máy tính, điện thoại di động, usb,...) ra khỏi phạm vi Công ty khi chưa được PSM/LM/UM phê duyệt:|
| |** Khi mang thiết bị Công ty đi công tác tại văn phòng của Sun* Inc, nhân viên cần phải đảm bảo tuân thủ đầy đủ các yêu cầu về việc bảo vệ tài sản và bảo mật thông tin theo các quy định của Công ty.*|
| |** Khi muốn mang thiết bị Công ty về nhà làm việc hoặc đi công tác ngoài phạm vi văn phòng của Sun* Inc, nhân viên phải nhận được sự đồng ý của của PSM/LM/UM phụ trách & hoàn tất các thủ tục tại ban Thư ký ISO.*|
| |*[Tham khảo: Sổ tay hướng dẫn thực hiện Quy định ANTT (mục 2.2)](https://docs.google.com/spreadsheets/d/1WFVi_bGu2FpzBJXx6MVGsGwlHVgUrdpgQutXoKg9O2U/edit#gid=1362637204)*|
|**PHẦN IV <br>QUY ĐỊNH VỀ QUẢN LÝ TRUY CẬP <br>(Áp dụng cho việc truy cập mạng và dịch vụ mạng; truy cập hệ thống, dữ liệu,...)**|**PHẦN IV <br>QUY ĐỊNH VỀ QUẢN LÝ TRUY CẬP <br>(Áp dụng cho việc truy cập mạng và dịch vụ mạng; truy cập hệ thống, dữ liệu,...)**|
| |Không được phép tự phát wifi khi sử dụng mạng của Công ty.|
| |Không được phép kết nối các thiết bị của cá nhân/ khách hàng (laptop, di động,...) vào mạng nội bộ của Công ty khi chưa được sự cho phép của PSM/LM/UM phụ trách & hoàn tất các thủ tục tại ban Thư ký ISO|
| |*[Tham khảo: Sổ tay hướng dẫn thực hiện Quy định ANTT (mục 1.3) ](https://docs.google.com/spreadsheets/d/1WFVi_bGu2FpzBJXx6MVGsGwlHVgUrdpgQutXoKg9O2U/edit#gid=1683192620)*|
| |Nhân viên muốn đăng nhập vào hệ thống Server Công ty, cần có sự cho phép của PSM/LM/UM|
| |[Tham khảo: Sổ tay hướng dẫn thực hiện Quy định ANTT (mục 3.1)](https://docs.google.com/spreadsheets/d/1WFVi_bGu2FpzBJXx6MVGsGwlHVgUrdpgQutXoKg9O2U/edit)|
| |Khi muốn làm việc, truy cập từ xa (VPN), nhân viên cần có sự cho phép của PSM/LM/UM và hoàn tất các thủ tục tại ban Thư ký ISO|
| |[Tham khảo: Sổ tay hướng dẫn thực hiện Quy định ANTT (mục 3.2)](https://docs.google.com/spreadsheets/d/1WFVi_bGu2FpzBJXx6MVGsGwlHVgUrdpgQutXoKg9O2U/edit)|
|**PHẦN V <br>QUY ĐỊNH VỀ MÃ HÓA TRONG KHI LÀM VIỆC**|**PHẦN V <br>QUY ĐỊNH VỀ MÃ HÓA TRONG KHI LÀM VIỆC**|
| |Đảm bảo mật khẩu đăng nhập bao gồm các yêu cầu sau:<br>- Phải thay đổi mật khẩu người dùng trong lần đăng nhập lần đầu tiên;<br>- Độ dài: tối thiểu 8 ký tự, bao gồm ký tự hoa, ký tự thường và số;<br>- Không được để trống hoặc lặp lại tài khoản người dùng, không sử dụng mật khẩu dễ nhớ hoặc dễ đoán như là tên hoặc ngày sinh;<br>- Phải được thay đổi định kỳ, 3 tháng/lần để bảo đảm an toàn;|
| |Không sử dụng cùng một mật khẩu đăng nhập cho nhiều hệ thống (Chỉ sử dụng cùng một mật khẩu cho nhiều hệ thống/dịch vụ nếu có thể đảm bảo mức độ bảo mật thỏa đáng cho mỗi hệ thống, dịch vụ);|
| |Không lưu mật khẩu ở trạng thái tự động đăng nhập, không lưu trên giấy, file script hoặc các phím chức năng;|
| |Không tiết lộ ID, mật khẩu của nhân viên cho bất cứ ai. Không để lộ mật khẩu vào các mạng nội bộ của Công ty; |
| |Trường hợp gửi email nếu có kèm tệp tin quan trọng (cần bảo mật), phải nén và gửi mật khẩu trong email sau;|
|**PHẦN VI <br>QUY ĐỊNH ĐỐI VỚI KHÁCH HÀNG **|**PHẦN VI <br>QUY ĐỊNH ĐỐI VỚI KHÁCH HÀNG **|
| |Khách hàng khi đến làm việc tại công ty sẽ do PSM dự án chịu trách nhiệm đón tiếp. <br>Tùy thuộc vào tình hình thực tế, UM phụ trách sẽ quyết định xem Khách hàng có cần thiết ký "Cam kết bảo mật dành cho Khách hàng làm việc tại công ty";<br>Trong đó lưu ý 2 trường hợp bắt buộc phải ký: <br>1. Khách hàng làm việc dài ngày (thường >=3 ngày)<br>2. Khách hàng sử dụng mạng nội bộ Công ty.|
| |Khách đến Công ty làm việc sẽ được cấp thẻ khách. <br>Nhân viên phụ trách tiếp đón khách liên hệ PA- phòng quản lý lao động đăng ký thủ tục nhận, trả thẻ khách;|
| |Khách hàng đến làm việc tại Công ty được phép sử dụng Wifi Customer.<br>Khách hàng chỉ được sử dụng mạng nội bộ Công ty (mạng dây/ wifi Staff) khi nhận được sự phê duyệt của PSM/LM và ký "Cam kết bảo mật dành cho Khách hàng làm việc tại công ty"|
| |Khách hàng, nhà cung cấp đến làm việc Công ty không được tự ý quay phim, chụp ảnh tại các khu vực: phòng Server, khu vực làm việc của nhân viên khi chưa nhận được sự cho phép, sự có mặt của người phụ trách đi cùng.|
| |*Để hỗ trợ thực hiện các Quy định đối với khách hàng, nhân viên phụ trách tham khảo theo hướng dẫn dưới đây:*|
| |*[Tham khảo: Sổ tay hướng dẫn thực hiện Quy định ANTT (mục 5) ](https://docs.google.com/spreadsheets/d/1WFVi_bGu2FpzBJXx6MVGsGwlHVgUrdpgQutXoKg9O2U/edit#gid=892547403)*|
"""

In [25]:
# Get list of chunks
list_chunks = split_table_into_chunks(table_markdown=big_table)
print(len(list_chunks))

7


In [26]:
# Get each preivious contents
lst_global_context = ["No Privious Contents"]
for chunk in list_chunks:
    user_prompt = USER_PROMPT_SUM.format(
        **{
            "file_name": "[ISMS_1-501_VN] 1. Các quy định An ninh thông tin",
            "previous_summary": lst_global_context[-1],
            "input_chunk": chunk
        }
    )
    system_prompt = SYSTEM_PROMPT_SUM
    message = [
        {
            "role": "system", 
            "content": system_prompt
        },
        {
            "role": "user", 
            "content": user_prompt
            },
    ]
    response = client.chat.completions.create(
        model=deployment_name,
        messages=message,
        temperature=0.0,
    )
    summary = response.choices[0].message.content
    lst_global_context.append(summary)

In [27]:
print(len(lst_global_context))

8


In [28]:

DEFAULT_ENTITY_TYPES = ["organization", "person", "geo", "event"]
DEFAULT_TUPLE_DELIMITER = "<|>"
DEFAULT_RECORD_DELIMITER = "##"
DEFAULT_COMPLETION_DELIMITER = "<|COMPLETE|>"


ENTITY_TYPES = [
    "System",
    "Asset",
    "Service",
    "Product",
    "Code",
    "Standard",
    "Organization",
    "Allowance",
    "Process",
    "Time",
    "Location",
    "Person",
    "Section",
    "Regulation",
    "Role",
    "Event",
    "Department",
    "Benefit",
    "Instruments",
    "Form",
]

In [29]:
lst_entities = []

for index, table_chunk in enumerate(list_chunks):
    user_prompt = USER_PROMPT.format(
        **{
            "entity_types": ENTITY_TYPES,
            "tuple_delimiter": DEFAULT_TUPLE_DELIMITER,
            "completion_delimiter": DEFAULT_COMPLETION_DELIMITER,
            "record_delimiter": DEFAULT_RECORD_DELIMITER,
            "global_context": list_chunks[index],
            "input_text": table_chunk,
        }
    )
    message = [
        {
            "role": "user", 
            "content": user_prompt
        },
    ]
    response = client.chat.completions.create(
        model=deployment_name,
        messages=message,
        temperature=0.0,
    )
    entities = response.choices[0].message.content
    lst_entities.append(entities)
    file_path = "test_extraction.txt"  # File name

    with open(file_path, "a", encoding="utf-8") as file:
        file.write(entities + "\n")
        file.write("======================= \n")  

In [30]:
def append_to_xlsx(file_path: str, table_chunks: list[str], entities: list[str]) -> None:
    """
    Append extracted entities and corresponding table chunks to an Excel file.

    Parameters:
    file_path (str): Path to the Excel file where data will be stored.
    table_chunks (list[str]): A list of input text chunks.
    entities (list[str]): A list of extracted entities corresponding to the input text chunks.

    If the file does not exist, a new Excel file is created. Otherwise, data is appended while preserving previous entries.
    """
    data = {"Input Text": table_chunks, "Extracted Entities": entities}
    df_new = pd.DataFrame(data)
    
    if not os.path.exists(file_path):
        df_new.to_excel(file_path, index=False)
    else:
        with pd.ExcelWriter(file_path, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
            df_existing = pd.read_excel(file_path)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined.to_excel(writer, index=False)

In [31]:
append_to_xlsx(file_path="table_extraction.xlsx", table_chunks=list_chunks, entities=lst_entities)